In [2]:
!pip install -q gliner kiwipiepy bm25s networkx scipy sentence-transformers llama-index llama-index-embeddings-huggingface llama-index-llms-huggingface python-mecab-ko faiss-cpu llama-index-vector-stores-faiss

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 MB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.9/151.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.6/579.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.

In [3]:
# 코랩에서 작업할 때는 아래 셀 주석 해제해 주세요.
# import os
# # 🛑 [중요] 욕심쟁이 프레임워크들의 메모리 점유 방지 (Import 전에 실행 필수!)
# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"       # TensorFlow가 필요할 때만 메모리 쓰게 함
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"  # JAX가 메모리 미리 찜하는 거 금지
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".10"   # 혹시 쓰더라도 10%만 쓰게 제한

from datasets import load_from_disk
import os
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from typing import List, Dict, Callable, Optional
import numpy as np
import json
import tqdm
from functools import partial

# LlamaIndex 관련
from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import QueryFusionRetriever, BaseRetriever
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.schema import TextNode, NodeWithScore, QueryBundle, BaseNode
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

# 기타 라이브러리
import bm25s
from gliner import GLiNER
from sentence_transformers import CrossEncoder

# Hugging Face 로그인
print("🚀 환경 설정 시작!")
HF_TOKEN = ""
os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
login(token=HF_TOKEN)

print(f"GPU 사용 가능 여부: {torch.cuda.is_available()}")

🚀 환경 설정 시작!
GPU 사용 가능 여부: True


In [6]:
# wiki data load
with open('/data/wikipedia_documents.json') as f:
    wiki_data = json.load(f)
id_to_title = {v["document_id"]: v["title"] for v in wiki_data.values()}


train_set_dir = "data/train_dataset"
dataset = load_from_disk(train_set_dir)

In [7]:
# --- 모델 로드 설정 ---
GEMMA_MODEL_NAME = "google/gemma-3-4b-it"  # 메모리 효율성을 위해 4b 대신 9b를 예시로 사용 (사용자 환경에 따라 변경 가능)
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
RERANKER_MODEL_NAME = "BAAI/bge-reranker-v2-m3"

In [8]:
# --- LLM 및 Tokenizer 로드 ---
def load_gemma():
    """Gemma 모델과 토크나이저를 로드합니다."""
    # Q: Gemma 3-4b-it 사용 예정이었는데, 현재는 Gemma 2-9b-it을 사용하려 합니다.
    # A: VRAM 상황에 따라 모델 이름을 적절히 변경하여 사용하세요.
    tokenizer = AutoTokenizer.from_pretrained(GEMMA_MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        GEMMA_MODEL_NAME,
        device_map="cuda" if torch.cuda.is_available() else "cpu",
        torch_dtype=torch.bfloat16,
    )
    return tokenizer, model

In [9]:
documents: List[Document] = []
for doc_id, data in wiki_data.items():
    # 'text' 필드를 문서 내용으로 사용
    documents.append(
        Document(
            text=data['text'],
            metadata={
                "document_id": data['document_id'],
                "title": data['title'],
                "corpus_source": data['corpus_source']
            }
        )
    )

In [10]:
# 만약 sentence split 이 완료되어 node.pkl 이 있다면 아래 셀로 실행해 주세요.
# import pickle
# import os
# from google.colab import drive


# # 2. 파일 경로 설정
# save_path = "my_search_index"
# node_file_path = os.path.join(save_path, "nodes.pkl")

# # 3. 불러오기 (Load)
# if os.path.exists(node_file_path):
#     print(f" '{node_file_path}' 파일을 로드 중입니다...")

#     with open(node_file_path, "rb") as f:
#         nodes = pickle.load(f)

#     print(f" 로드 완료! 총 {len(nodes)}개의 청크(Node)가 복구되었습니다.")

#     # 4. 잘 불러와졌는지 눈으로 확인 (첫 번째 노드 내용)
#     # (에러 없이 내용이 출력되면 성공입니다!)
#     try:
#         print(f" 첫 번째 노드 내용: {nodes[0].text[:50]}...")
#     except:
#         print(" 내용 확인 실패 (형식이 다를 수 있음)")

# else:
#     print(f" 파일이 없습니다! 경로를 다시 확인해주세요: {node_file_path}")

📂 '/content/drive/MyDrive/vllm_baseline/my_search_index/nodes.pkl' 파일을 로드 중입니다...
✅ 로드 완료! 총 373436개의 청크(Node)가 복구되었습니다.
👀 첫 번째 노드 내용: 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태...


In [ ]:
# 3. 문서 청킹 (Node 생성)
# SentenceSplitter는 문장 단위 분할을 기본으로 하면서,
# 최종 청크 크기를 chunk_size=512로 제한합니다.
splitter = SentenceSplitter(chunk_size=256, chunk_overlap=128)

# nodes에는 작은 텍스트 청크(TextNode)들이 리스트 형태로 담깁니다.
nodes: List[TextNode] = splitter.get_nodes_from_documents(documents)

print(f"원본 문서 개수: {len(documents)}개")
print(f"생성된 청크(Node) 개수: {len(nodes)}개")
print(f"첫 번째 청크 텍스트 예시: {nodes[0].get_content()[:100]}...")

원본 문서 개수: 60613개
생성된 청크(Node) 개수: 373436개
첫 번째 청크 텍스트 예시: 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.

이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.

# 첫 번째 부...


In [12]:
embed_model = HuggingFaceEmbedding(
    model_name=EMBEDDING_MODEL_NAME,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [13]:
# faiss vs

dummy_emb = embed_model.get_text_embedding("dim 체크용")
dim = len(dummy_emb)
faiss_index = faiss.IndexFlatIP(dim)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [15]:
vector_index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model,
)

KeyboardInterrupt: 

In [17]:
class Reranker:
    def __init__(self, model_name: str = "BAAI/bge-reranker-v2-m3"): # 모델명은 쓰시는걸로
        self.model = CrossEncoder(model_name, device="cuda" if torch.cuda.is_available() else "cpu")

    def rerank(self, query: str, docs: List[str], doc_ids: List[str], top_k: int = 5):
        if not docs: return [], []



        # 만약 ID가 전부 -1 이거나 None이면 여기가 문제임!
        unique_ids = set(doc_ids)


        # 1. 점수 계산
        pairs = [[query, d] for d in docs]
        scores = self.model.predict(pairs)

        # 2. 정렬
        combined = list(zip(scores, docs, doc_ids))
        combined.sort(key=lambda x: x[0], reverse=True)

        # 3. 중복 제거
        seen_ids = set()
        final_results = []
        final_ids = []

        for score, text, did in combined:
            # ID가 -1이나 N/A면 중복 제거 하지 말고 그냥 다 넣어버리기 (임시 조치)
            if did == -1 or did == 'N/A' or did is None:
                final_results.append((text, score))
                final_ids.append(did)

            # 정상 ID인 경우에만 중복 제거 수행
            elif did not in seen_ids:
                seen_ids.add(did)
                final_results.append((text, score))
                final_ids.append(did)

            if len(final_results) >= top_k:
                break


        return final_results, final_ids

In [18]:
reranker = Reranker()


config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
# tokenizer, model = load_gemma()
# gemma_llm = HuggingFaceLLM(
#     # model_name을 지정할 필요가 없거나, 명시적으로 지정해도 model/tokenizer 인자가 우선됩니다.
#     model=model,        # 이미 로드된 PyTorch 모델 객체
#     tokenizer=tokenizer,  # 이미 로드된 Tokenizer 객체
#     device="cuda" if torch.cuda.is_available() else "cpu",
#     # device_map="auto" 등은 이미 model 로드 시 적용되었으므로 LlamaIndex LLM에서는 불필요

#     # 템플릿 처리 방식 등 LlamaIndex 관련 설정만 추가
#     context_window=8192, # 예시: Gemma의 Context Window 설정 (필요에 따라)
# )

# # 3. LlamaIndex 설정에 적용
# Settings.llm = gemma_llm

In [19]:
print("🔥 GLiNer XL 모델 로딩 중...")

# 1. 라벨 정의
GLINER_TAG = {
    "언어": {"CV_LANGUAGE": "문명_언어", "TR_HUMANITIES": "이론_철학/언어/역사"}, "문학": {"FD_HUMANITIES": "학문 분야_인문학", "AFA_DOCUMENT": "인공물_도서/서적 작품명"}, "역사": {"CV_CULTURE": "문명_문명/문화", "AF_CULTURAL_ASSET": "인공물_문화재", "DT_DYNASTY": "날짜_왕조시대", "DT_GEOAGE": "날짜_지질시대", "EV_WAR_REVOLUTION": "사건_전쟁/혁명", "EV_OTHERS": "사건_기타"}, "철학": {"TR_HUMANITIES": "이론_철학/언어/역사", "CV_TRIBE": "문명_민족/종족"}, "교육": {"OGG_EDUCATION": "기관_교육", "OGG_LIBRARY": "기관_도서관"}, "민속": {"CV_CULTURE": "문명_문명/문화", "EV_FESTIVAL": "사건_축제/영화제"}, "인문 일반": {"FD_HUMANITIES": "학문 분야_인문학"},
    "법률": {"CV_LAW": "문명_법/법률", "OGG_LAW": "기관_법률", "CV_POLICY": "문명_제도/정책"}, "군사": {"OGG_MILITARY": "기관_군사", "AF_WEAPON": "인공물_무기"}, "경영": {"OGG_ECONOMY": "기관_경제", "AFW_SERVICE_PRODUCTS": "인공물_서비스 상품"}, "경제": {"OGG_ECONOMY": "기관_경제", "CV_CURRENCY": "문명_통화", "CV_TAX": "문명_조세"}, "복지": {"CV_FUNDS": "문명_연금/기금"}, "정치": {"OGG_POLITICS": "기관_정부/공공", "CV_POLICY": "문명_제도/정책", "EV_ACTIVITY": "사건_사회운동/선언"}, "매체": {"OGG_MEDIA": "기관_미디어", "AFA_VIDEO": "인공물_영화/TV 프로그램"}, "행정": {"OGG_POLITICS": "기관_정부/공공"}, "심리": {"FD_SOCIAL_SCIENCE": "학문 분야_사회과학"}, "사회 일반": {"FD_SOCIAL_SCIENCE": "학문 분야_사회과학"},
    "지구": {"FD_SCIENCE": "학문 분야_과학", "MT_ROCK": "물질_암석"}, "지리": {"LC_OTHERS": "장소_기타", "LCG_MOUNTAIN": "장소_산/산맥", "LCG_RIVER": "장소_강/호수", "LCG_OCEAN": "장소_바다", "LCG_ISLAND": "장소_섬", "LCG_CONTINENT": "장소_대륙", "TM_DIRECTION": "용어_방향"}, "해양": {"LCG_OCEAN": "장소_바다", "LCG_BAY": "장소_반도/만"}, "천문": {"LC_SPACE": "장소_천체", "FD_SCIENCE": "학문 분야_과학"}, "환경": {"TM_CLIMATE": "용어_기후 지역", "FD_SCIENCE": "학문 분야_과학"}, "생명": {"TM_CELL_TISSUE_ORGAN": "용어_세포/조직/기관", "FD_SCIENCE": "학문 분야_과학"}, "동물": {"AM_INSECT": "동물_곤충", "AM_BIRD": "동물_조류", "AM_FISH": "동물_어류", "AM_MAMMALIA": "동물_포유류", "AM_AMPHIBIA": "동물_양서류", "AM_REPTILIA": "동물_파충류", "AM_TYPE": "동물_분류명", "AM_PART": "동물_부위명", "AM_OTHERS": "동물_기타"}, "식물": {"PT_FLOWER": "식물_꽃", "PT_GRASS": "식물_풀", "PT_TYPE": "식물_분류명", "PT_PART": "식물_부위명", "PT_OTHERS": "식물_기타", "PT_TREE": "식물_나무", "PT_FRUIT": "식물_과일/열매"}, "천연자원": {"MT_ELEMENT": "물질_원소"}, "수학": {"FD_SCIENCE": "학문 분야_과학", "TM_SHAPE": "용어_모양/형태", "QT_SIZE": "수량_넓이/면적", "QT_LENGTH": "수량_길이/거리", "QT_VOLUME": "수량_부피", "QT_PERCENTAGE": "수량_백분율"}, "물리": {"FD_SCIENCE": "학문 분야_과학", "TR_SCIENCE": "이론_과학", "QT_SPEED": "수량_속도", "QT_TEMPERATURE": "수량_온도", "QT_WEIGHT": "수량_무게"}, "화학": {"MT_CHEMICAL": "물질_화학", "MT_ELEMENT": "물질_원소", "MT_METAL": "물질_금속"}, "자연 일반": {"FD_SCIENCE": "학문 분야_과학"},
    "농업": {"PT_FRUIT": "식물_과일/열매", "FD_OTHERS": "학문 분야_기타"}, "수산업": {"AM_FISH": "동물_어류"}, "임업": {"PT_TREE": "식물_나무"}, "광업": {"MT_ROCK": "물질_암석", "MT_METAL": "물질_금속"}, "공업": {"AFW_OTHER_PRODUCTS": "인공물_기타 상품"}, "서비스업": {"AFW_SERVICE_PRODUCTS": "인공물_서비스 상품", "OGG_HOTEL": "기관_숙박 업체"}, "산업 일반": {"OGG_ECONOMY": "기관_경제"},
    "의학": {"FD_MEDICINE": "학문 분야_의학", "TR_MEDICINE": "이론_의학", "TMM_DISEASE": "용어_증상/질병"}, "약학": {"TMM_DRUG": "용어_약품"}, "한의": {"FD_MEDICINE": "학문 분야_의학"}, "수의": {"FD_MEDICINE": "학문 분야_의학"}, "식품": {"CV_FOOD": "문명_음식", "CV_DRINK": "문명_음료/술", "CV_FOOD_STYLE": "문명_음식 유형"}, "보건 일반": {"OGG_MEDICINE": "기관_의료"},
    "건설": {"AF_BUILDING": "인공물_건축물/토목건설물", "CV_BUILDING_TYPE": "문명_건축 양식"}, "교통": {"AF_TRANSPORT": "인공물_교통수단/운송수단", "AF_ROAD": "인공물_도로/철로"}, "기계": {"TMI_HW": "용어_IT 하드웨어"}, "전기·전자": {"TMI_HW": "용어_IT 하드웨어"}, "재료": {"MT_ELEMENT": "물질_원소"}, "정보·통신": {"TMI_SW": "용어_IT 소프트웨어", "TMI_HW": "용어_IT 하드웨어", "TMI_SITE": "용어_URL 주소", "TMI_EMAIL": "용어_이메일 주소", "TMI_MODEL": "용어_제품 모델명", "TMI_SERVICE": "용어_IT 서비스", "TMI_PROJECT": "용어_프로젝트"}, "공학 일반": {"FD_SCIENCE": "학문 분야_과학"},
    "체육": {"CV_SPORTS": "문명_스포츠", "OGG_SPORTS": "기관_스포츠", "CV_SPORTS_POSITION": "문명_스포츠 포지션", "CV_SPORTS_INST": "문명_스포츠 용품/도구", "EV_SPORTS": "사건_스포츠 행사", "TM_SPORTS": "용어_스포츠"}, "연기": {"AFA_PERFORMANCE": "인공물_춤/공연/연극 작품명"}, "영상": {"AFA_VIDEO": "인공물_영화/TV 프로그램"}, "무용": {"AFA_PERFORMANCE": "인공물_춤/공연/연극 작품명"}, "음악": {"AFA_MUSIC": "인공물_음악 작품명", "AF_MUSICAL_INSTRUMENT": "인공물_악기", "OGG_ART": "기관_예술"}, "미술": {"AFA_ART_CRAFT": "인공물_미술/조형 작품명", "FD_ART": "학문 분야_예술", "TM_COLOR": "용어_색깔"}, "복식": {"CV_CLOTHING": "문명_의복/섬유"}, "공예": {"AFA_ART_CRAFT": "인공물_미술/조형 작품명"}, "예체능 일반": {"FD_ART": "학문 분야_예술"},
    "가톨릭": {"OGG_RELIGION": "기관_종교"}, "기독교": {"OGG_RELIGION": "기관_종교"}, "불교": {"OGG_RELIGION": "기관_종교"}, "종교 일반": {"OGG_RELIGION": "기관_종교"},
    "인명": {"PS_NAME": "인물_사람", "PS_CHARACTER": "인물_가상 캐릭터", "CV_OCCUPATION": "문명_직업", "CV_POSITION": "문명_직위/직책", "CV_RELATION": "문명_가족/친족 관계"}, "지명": {"LCP_COUNTRY": "장소_국가", "LCP_PROVINCE": "장소_도/주 지역", "LCP_COUNTY": "장소_세부 행정구역", "LCP_CITY": "장소_도시", "LCP_CAPITALCITY": "장소_수도", "LC_OTHERS": "장소_기타"}, "책명": {"AFA_DOCUMENT": "인공물_도서/서적 작품명"}, "고유명 일반": {"OGG_OTHERS": "기관_기타"}
}
gliner_labels = []

for middle_cat, inner_map in GLINER_TAG.items():
    for gliner_code, leaf_label in inner_map.items():
        gliner_labels.append(leaf_label)
gliner_labels = sorted(list(set(gliner_labels)))

# 2. GLiNer XL 모델 로드
gliner_model = GLiNER.from_pretrained("lots-o/gliner-bi-ko-xlarge-v1")
gliner_model = gliner_model.to("cuda")
print("✅ GLiNer XL 모델 로드 완료!")

🔥 GLiNer XL 모델 로딩 중...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/5.50G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/gliner/model.py:418: UserWarning: Resizing embeddings is not supported for bi-encoder models.
  instance.resize_embeddings()


✅ GLiNer XL 모델 로드 완료!


In [20]:
from functools import partial
import bm25s
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore, QueryBundle, BaseNode
from typing import List, Dict, Callable, Optional
import tqdm

# -----------------------------------------------------------------------------
# 3. [핵심] 배치 지원 토크나이저 함수 (List[str] 입력을 받음)
# -----------------------------------------------------------------------------
def tokenize_gliner_batch(
    texts: List[str],  # 🚨 입력이 단일 str이 아니라 List[str]입니다!
    model: GLiNER,
    labels: List[str],
    label_chunk_size: int = 20,
    score_threshold_ratio: float = 1.05
) -> List[List[str]]:
    """
    여러 문장을 한 번에 받아서, 각 문장별 토큰 리스트를 반환합니다.
    (V100 GPU 병렬 처리를 위해 필수)
    """
    if not texts: return []

    # 결과 저장용 (문서 개수만큼 빈 리스트 생성)
    batch_results = [[] for _ in texts]

    # 라벨 배치 처리 (Label Chunking)
    for i in range(0, len(labels), label_chunk_size):
        sub_labels = labels[i : i + label_chunk_size]
        try:
            # 🚀 model.batch_predict_entities 사용 (속도 향상의 핵심)
            batch_preds = model.batch_predict_entities(
                texts, sub_labels, flat_ner=True, threshold=0.1
            )
            # 결과 병합
            for doc_idx, entities in enumerate(batch_preds):
                batch_results[doc_idx].extend(entities)
        except Exception: continue

    # 후처리 (각 문서별로 점수 필터 & 문장 쪼개기 적용)
    final_batch_tokens = []

    for entities in batch_results:
        if not entities:
            final_batch_tokens.append([])
            continue

        # 1. 점수 필터링 (Relative Threshold)
        max_score = max(e['score'] for e in entities)
        cutoff_score = max_score / score_threshold_ratio
        filtered = [e for e in entities if e['score'] >= cutoff_score]

        # 2. 스마트 필터 (문장 쪼개기)
        doc_tokens = set()
        for e in filtered:
            token_text = e['text']
            # 띄어쓰기 2개 이상(3어절)이면 쪼개기
            if token_text.count(' ') >= 2:
                for t in token_text.split():
                    doc_tokens.add(t)
            else:
                doc_tokens.add(token_text)

        final_batch_tokens.append(list(doc_tokens))

    return final_batch_tokens


class GLiNerBM25Retriever(BaseRetriever):
    """partial 토크나이저를 받아서 배치 처리하는 Retriever"""
    def __init__(
        self,
        nodes: List[BaseNode],
        tokenizer: Callable[[List[str]], List[List[str]]], # 🚨 배치 토크나이저 시그니처
        similarity_top_k: int = 30,
        doc_batch_size: int = 64 # 🚀 문서 배치 사이즈 (한 번에 처리할 문서 수)
    ) -> None:

        self._nodes = nodes
        self._similarity_top_k = similarity_top_k
        self._tokenizer = tokenizer
        self.doc_batch_size = doc_batch_size

        print(f"🚀 GLiNer Batch Indexing... Docs: {len(nodes)}, Batch: {doc_batch_size}")

        corpus_tokens = []

        # 문서를 뭉텅이(Batch)로 잘라서 토크나이저 함수 호출
        for i in tqdm.tqdm(range(0, len(nodes), doc_batch_size), desc="Indexing"):
            batch_nodes = nodes[i : i + doc_batch_size]
            batch_texts = [n.text if n.text else "" for n in batch_nodes]

            # 여기서 partial로 만든 함수에 '리스트'를 던집니다!
            batch_tokens_list = self._tokenizer(batch_texts)
            corpus_tokens.extend(batch_tokens_list)

        self._bm25 = bm25s.BM25()
        self._bm25.index(corpus_tokens)

        print("✅ Indexing Complete!")
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query = query_bundle.query_str

        # 쿼리는 1개지만, 배치 함수니까 리스트로 감싸서 보냄
        # 결과도 리스트의 리스트니까 [0]으로 꺼냄
        query_tokens = self._tokenizer([query])[0]

        if not query_tokens: return []

        tokenized_query = [query_tokens]
        actual_k = min(self._similarity_top_k, len(self._nodes))
        if actual_k == 0: return []

        results, scores = self._bm25.retrieve(tokenized_query, k=actual_k)

        nodes_with_scores: List[NodeWithScore] = []
        for idx, score in zip(results[0], scores[0]):
            if score > 0:
                nodes_with_scores.append(
                    NodeWithScore(node=self._nodes[idx], score=float(score))
                )
        return nodes_with_scores

    def persist(self, path: str):
        self._bm25.save(path)
        print(f"✅ 인덱스 저장 완료: {path}")

In [ ]:
# 1. 배치 토크나이저 함수를 partial로 조립
my_gliner_batch_tokenizer = partial(
    tokenize_gliner_batch,
    model=gliner_model,         # XL 모델
    labels=gliner_labels,              # 150개 라벨
    label_chunk_size=50,        # 라벨 20개씩 끊기
    score_threshold_ratio=2.5  # 점수 필터
)

# 2. Retriever 생성 (인덱싱 시작)
# nodes는 이전 단계에서 이미 생성되어 있어야 합니다!
gliner_bm25_retriever = GLiNerBM25Retriever(
    nodes=nodes,
    similarity_top_k=30,
    tokenizer=my_gliner_batch_tokenizer, # partial 함수 전달
    doc_batch_size=32 # 🚀 한 번에 64개씩 처리 (V100 메모리 활용)
)

print("✅ GLiNer Retriever 준비 완료 (Batch Mode)")

save_dir = "gliner_bm25_index_v1"
gliner_bm25_retriever._bm25.save(save_dir)
print(f"💾 인덱스를 '{save_dir}' 폴더에 안전하게 저장했습니다.")

🚀 GLiNer Batch Indexing... Docs: 373436, Batch: 32


Indexing:   0%|          | 0/11670 [00:00<?, ?it/s]/tmp/ipykernel_35058/3565486690.py:32: FutureWarning: GLiNER.batch_predict_entities is deprecated and will be removed in a future release. Please use GLiNER.inference instead.
  batch_preds = model.batch_predict_entities(
Indexing: 100%|██████████| 11670/11670 [8:48:28<00:00,  2.72s/it]  


BM25S Create Vocab:   0%|          | 0/373436 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/373436 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/373436 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/373436 [00:00<?, ?it/s]

✅ Indexing Complete!
✅ GLiNer Retriever 준비 완료 (Batch Mode)
💾 인덱스를 'gliner_bm25_index_v1' 폴더에 안전하게 저장했습니다.


In [28]:
# 토크나이징 결과를 저장했다면 아래 셀을 사용해주세요.
# # 토크나이저 함수 다시 조립 (설정값 동일하게!)
# my_gliner_batch_tokenizer = partial(
#     tokenize_gliner_batch,
#     model=gliner_model,
#     labels=gliner_labels,
#     label_chunk_size=50,
#     score_threshold_ratio=2.5
# )

# # =========================================================
# # 2. 저장된 데이터 불러오기 (Nodes & Index)
# # =========================================================
# save_dir = "gliner_bm25_index_v1" # 아까 저장한 폴더

# print(f"📂 2. '{save_dir}'에서 데이터 복구 중...")

# # [중요] Nodes(본문)도 불러와야 합니다.
# # (만약 nodes를 pickle로 저장을 안 하셨다면, nodes 생성 코드는 다시 돌려야 합니다 ㅠㅠ)
# # nodes 저장 예시: with open("nodes.pkl", "wb") as f: pickle.dump(nodes, f)
# if os.path.exists("nodes.pkl"):
#     with open("nodes.pkl", "rb") as f:
#         loaded_nodes = pickle.load(f)
# else:
#     print("⚠️ 경고: 'nodes.pkl'이 없습니다. nodes 변수가 메모리에 있어야 합니다.")
#     # nodes = ... (여기서 nodes를 다시 만들거나 가져와야 함)
#     loaded_nodes = nodes # (이미 메모리에 있다면)

# # BM25 인덱스 파일 로드 (여기가 핵심! 0초 걸림)
# # load_corpus=False: 우리는 nodes를 따로 쓰므로 텍스트까지 로드할 필요 없음
# loaded_bm25 = bm25s.BM25.load(save_dir, load_corpus=False)

# # =========================================================
# # 3. [핵심기술] 검색기 조립 (수술 집도 👨‍⚕️)
# # =========================================================
# print("🔧 3. 검색기 조립 중 (인덱싱 과정 스킵)...")

# # 1) 클래스 껍데기만 생성 (__new__ 사용 -> __init__ 실행 안 됨 -> 인덱싱 안 함)
# restored_retriever = GLiNerBM25Retriever.__new__(GLiNerBM25Retriever)

# # 2) 부품 수동 끼워 넣기
# restored_retriever._nodes = loaded_nodes      # 본문 연결
# restored_retriever._bm25 = loaded_bm25        # 뇌(인덱스) 연결
# restored_retriever._tokenizer = my_gliner_batch_tokenizer # 눈(토크나이저) 연결

# # 3) 기타 설정 복원
# restored_retriever._similarity_top_k = 30
# restored_retriever.doc_batch_size = 32
# restored_retriever._corpus_tokenizer = None # 이미 인덱싱 돼서 필요 없음

# print("🎉 GLiNER 검색기 부활 완료! 바로 retrieve() 하세요.")

📂 2. '/content/drive/MyDrive/vllm_baseline/gliner_bm25_index_v1'에서 데이터 복구 중...
⚠️ 경고: 'nodes.pkl'이 없습니다. nodes 변수가 메모리에 있어야 합니다.
🔧 3. 검색기 조립 중 (인덱싱 과정 스킵)...
🎉 GLiNER 검색기 부활 완료! 바로 retrieve() 하세요.


In [ ]:
#골든리트리버
retriever = vector_index.as_retriever(similarity_top_k=50)



Settings.llm = None

print("🔗 Fusion Retriever 연결")

# 2. 리트리버 생성
fusion_retriever = QueryFusionRetriever(
    retrievers=[retriever, gliner_bm25_retriever], # 변수명 확인 필요 (final_retriever 일 수 있음)
    similarity_top_k=30,
    num_queries=1,            # 쿼리 생성 안 함
    use_async=False,
    mode="reciprocal_rerank",
    llm=None                  # 🛑 여기에도 명시적으로 None 넣어주면 더 확실합니다.
)

print("✅ Fusion Retriever 생성 완료!")

LLM is explicitly disabled. Using MockLLM.
🔗 Fusion Retriever 연결
✅ Fusion Retriever 생성 완료!


In [24]:
# 1. 테스트할 질문 가져오기
query = dataset['train'][307]['question']
print(f"❓ 질문: {query}")
print("-" * 60)

# 2. GLiNer 단독 검색 실행
gliner_results = restored_retriever.retrieve(query)

# 3. 결과 뜯어보기
print(f"\n🔍 [GLiNer Only] 검색 결과: {len(gliner_results)}개 발견\n")

for i, node_with_score in enumerate(gliner_results[:5]): # 상위 5개만
    node = node_with_score.node
    score = node_with_score.score

    # 🛠️ [수정] f-string 안에서 \n을 못 쓰므로, 밖에서 미리 처리합니다.
    clean_text = node.text[:100].replace('\n', ' ')

    print(f"🏅 [Rank {i+1}] BM25 Score: {score:.4f}")
    print(f"📄 문서 ID: {node.metadata.get('document_id', 'N/A')}")
    print(f"📝 내용: {clean_text}...") # 미리 만들어둔 변수 사용
    print("-" * 60)

❓ 질문: 남은 뉴잉글랜드초원멧닭들을 떼죽음으로 몰고 간 병은?
------------------------------------------------------------


/tmp/ipython-input-3565486690.py:32: FutureWarning: GLiNER.batch_predict_entities is deprecated and will be removed in a future release. Please use GLiNER.inference instead.
  batch_preds = model.batch_predict_entities(


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]


🔍 [GLiNer Only] 검색 결과: 30개 발견

🏅 [Rank 1] BM25 Score: 10.7300
📄 문서 ID: 24635
📝 내용: 다행히도, 그때쯤 이들을 보호하자는 여론이 조성되었다. 648만 평방미터에 달하는 섬은 보호구로 지정했다. 효과는 뛰어나서 뉴잉글랜드초원멧닭의 개체수가 1915년에는 800마리, ...
------------------------------------------------------------
🏅 [Rank 2] BM25 Score: 9.9924
📄 문서 ID: 24636
📝 내용: "모든 날씨 변화와 질병, 천적의 영향을 받는 이 새가 그렇게 오랫동안 홀로 살 수 있었던 것은 정말 놀랍다." 1931년 4월, 그로스는 이 홀로 남은 새를 잡아 띠를 두른 후 ...
------------------------------------------------------------
🏅 [Rank 3] BM25 Score: 8.2840
📄 문서 ID: 24634
📝 내용: 그 결과 뉴잉글랜드초원멧닭은 다른 두 아종처럼 과도한 남획으로 인해 큰 타격을 입어 19세기 중반에는 뉴잉글랜드 본토에서 멸종했다. 단지 매사추세츠 주 코드곶 인근해의 작은 섬인 ...
------------------------------------------------------------
🏅 [Rank 4] BM25 Score: 7.4364
📄 문서 ID: 24634
📝 내용: 개신교 신도들인 청교도들은 성서뿐만 아니라 총도 많이 들고 왔는데, 종교적 박해를 피해서 온 이들은 북아메리카에서는 박해자로 돌변하여 아메리카 토착민들과 야생동물들에 대한 박해를 ...
------------------------------------------------------------
🏅 [Rank 5] BM25 Score: 6.6121
📄 문서 ID: 24635
📝 내용: 효과는 뛰어나서 뉴잉글랜드초원멧닭의 개체수가 1915년에는 800마리, 1916년에는 2천 마

In [ ]:
query = dataset['train'][86]['question']
retrieved_nodes = fusion_retriever.retrieve(query)

docs_for_rerank = [n.node.text for n in retrieved_nodes]
ids_for_rerank = [n.node.metadata['document_id'] for n in retrieved_nodes]

reranked_results = reranker.rerank(query, docs_for_rerank, ids_for_rerank, top_k=5)

print(reranked_results)

final_context = "\n\n---\n\n".join([f"[{i+1}] {d[0]}" for i, d in enumerate(reranked_results[0])])
print(f'Q. {query}')

print(reranked_results[1])


/tmp/ipykernel_35058/3565486690.py:32: FutureWarning: GLiNER.batch_predict_entities is deprecated and will be removed in a future release. Please use GLiNER.inference instead.
  batch_preds = model.batch_predict_entities(


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

([('부유한 전당포 가문의 외동딸이었던 우메노는 에도에서 열린 마츠리에 나서던 도중에 잘 생긴 소년의 모습에 반하게 된다. 소년의 모습을 다시 보고 싶었던 우메노는 부모에게 시집을 가고 싶다고 말했지만 부모의 반대로 인해 좌절했고 음력 1월 18일에 상사병으로 인해 사망하고 만다.\\n\\n우메노의 부모는 딸의 결혼에 반대한 것을 크게 후회하면서 슬픔에 빠지게 된다.', np.float32(0.9926979)), ('그의 사인은 독살이 유력하나, 뇌전증으로 사망했다는 부검 기록 역시 존재한다. 장례식은 다음 일요일인 7월 2일에 열렸고, 모든 성직자들이 동원되었다. 신심회와 고위 귀족 125명이 상복과 함께 손에 횃불을 들었고, 장례 절차가 우르비노 시 사람들 전체 앞에서 이루어졌다.  그의 아버지는 권좌를 다시 잡았지만 포기하고 만다.', np.float32(0.08497089)), ('우라노스는 고환이 잘리자 고통을 참지 못하고 큰 비명을 지르면서 그 자리에 주저앉았다. 아들인 크로노스에게 거세당하면서 1인자에서 쫓겨나고 더 이상 가이아와 사랑을 나눌 수 없게 된 우라노스는 고통 속에서 많은 피를 흘리며 도망가서는 다신 가이아를 찾지 않았다.\\n\\n우라노스가 흘린 피가 땅에 떨어지면서 기가스, 에리니에스, 멜리아데스가 탄생하였다.', np.float32(0.0050825994)), ('1444년 7월 22일, 최근 교황 에우제니오 4세에 의해 우르비노 공작이 된 그의 배다른 형제 오단토니오 다 몬테펠트로가 페데리코가 가담했을 것이라고 추정되나 밝혀지지는 않은 음모 속에서 암살되었고, 그 뒤에 우르비노는 포위를 당하게 되었다. 하지만 소국의 경제 상황은 악화되고 있었지만, 그는 콘도티에로로서 전쟁을 벌였다.', np.float32(0.0016561096)), ("이 때 그는 이자나미가 이전과 같이 아름답지 않다는 것을 확인했고, 이 때 둘간의 짧은 언쟁이 오고갔으며, 이 때 이자나미는 매일 인간 천 명의 목숨을 빼앗을 것을 약속하였고 이로 인하여 죽음의

In [ ]:
docs_for_rerank = [n.node.text for n in retrieved_nodes]
ids_for_rerank = [n.node.metadata['document_id'] for n in retrieved_nodes]

reranked_results = reranker.rerank(query, docs_for_rerank, ids_for_rerank, top_k=5)

In [ ]:
valid_set_dir = "/data/ephemeral/home/data/train_dataset"
valid_dataset = load_from_disk(valid_set_dir)

In [ ]:
from tqdm.notebook import tqdm  # 코랩용 예쁜 진행바
import json
import numpy as np
import os

# 결과를 담을 리스트
result_for_test = []

print("🚀 Validation 데이터 전체 평가 시작...")

# 1. 데이터셋 루프
for i in tqdm(range(len(valid_dataset['validation']))):

    # 질문과 id 가져오기
    test_q_query = valid_dataset['validation'][i]['question']
    test_q_id = valid_dataset['validation'][i]['id']

    # 2. 골든리트리버 (Fusion 검색)
    retrieved_nodes_test = fusion_retriever.retrieve(test_q_query)

    # 3. Reranker용 데이터 준비
    docs_for_rerank_test = [n.node.text for n in retrieved_nodes_test]
    # metadata 안전하게 가져오기 (혹시 모를 에러 방지용 get 사용)
    ids_for_rerank_test = [n.node.metadata.get('document_id', -1) for n in retrieved_nodes_test]

    # 4. Rerank 실행 (중복 제거 로직 포함됨)
    # reranked_results_test[1] 에는 이미 중복이 제거된 ID 리스트가 들어있습니다.
    reranked_results_test = reranker.rerank(test_q_query, docs_for_rerank_test, ids_for_rerank_test, top_k=5)

    # 5. [핵심 수정] ID 추출 (Numpy 슬라이싱 삭제) 🛠️
    # 이미 1차원 리스트이므로 그냥 int로 변환만 하면 됩니다.
    final_doc_ids = [int(did) for did in reranked_results_test[1]]

    result_for_test.append([test_q_id, final_doc_ids])

# ---------------------------------------------------------
# JSON 변환 및 저장
# ---------------------------------------------------------
def convert_to_json(data):
    question_ids = []
    document_lists = []

    for q_id, doc_list in data:
        question_ids.append(q_id)
        document_lists.append(doc_list)

    result_dict = {
        "question_id": question_ids,
        "document_id": document_lists
    }
    return result_dict

# 변환
json_test = convert_to_json(result_for_test)

# 파일 경로 (요청하신 경로)
file_path = './document_list/validation_ner_hybrid.json'

# 폴더가 없으면 에러 나니까 생성해주는 코드 추가
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# 저장 (한글 깨짐 방지 포함)
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(json_test, f, ensure_ascii=False, indent=4)

print(f"💾 저장 완료! 경로: {file_path}")

🚀 Validation 데이터 전체 평가 시작...


  0%|          | 0/240 [00:00<?, ?it/s]

/tmp/ipykernel_35058/3565486690.py:32: FutureWarning: GLiNER.batch_predict_entities is deprecated and will be removed in a future release. Please use GLiNER.inference instead.
  batch_preds = model.batch_predict_entities(


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

...

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

💾 저장 완료! 경로: ./document_list/validation_ner_hybrid.json


In [ ]:
test_set_dir = "/data/ephemeral/home/data/test_dataset"
test_dataset = load_from_disk(test_set_dir)

In [ ]:
from tqdm.notebook import tqdm
import json
import numpy as np
import os

# 🚨 결과 담을 리스트 (이름 통일함)
final_test_results = []

print("🚀 Test 데이터셋 전체 평가 시작...")

# 1. 데이터셋 루프 (valid -> test 로 변경)
for i in tqdm(range(len(test_dataset['validation']))):

    # 질문과 id 가져오기 (test_dataset 사용)
    target_query = test_dataset['validation'][i]['question']
    target_id = test_dataset['validation'][i]['id']

    # 2. 골든리트리버 (Fusion 검색)
    retrieved_nodes = fusion_retriever.retrieve(target_query)

    # 3. Reranker용 데이터 준비
    docs_for_rerank = [n.node.text for n in retrieved_nodes]
    # metadata 안전하게 가져오기
    ids_for_rerank = [n.node.metadata.get('document_id', -1) for n in retrieved_nodes]

    # 4. Rerank 실행 (중복 제거 로직 포함)
    reranked_results = reranker.rerank(target_query, docs_for_rerank, ids_for_rerank, top_k=5)

    # 5. ID 추출 (reranked_results[1]이 이미 정제된 ID 리스트임)
    final_doc_ids = [int(did) for did in reranked_results[1]]

    # 🚨 위에서 선언한 리스트에 정확히 append
    final_test_results.append([target_id, final_doc_ids])

# ---------------------------------------------------------
# JSON 변환 및 저장
# ---------------------------------------------------------
def convert_to_json(data):
    question_ids = []
    document_lists = []

    for q_id, doc_list in data:
        question_ids.append(q_id)
        document_lists.append(doc_list)

    result_dict = {
        "question_id": question_ids,
        "document_id": document_lists
    }
    return result_dict

# 변환 (올바른 리스트 전달)
json_output = convert_to_json(final_test_results)

# 파일 경로
file_path = './document_list/test_ner_hybrid.json'

# 폴더 생성 (안전장치)
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# 저장
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(json_output, f, ensure_ascii=False, indent=4)

print(f"💾 Test 결과 저장 완료! 경로: {file_path}")

🚀 Test 데이터셋 전체 평가 시작...


  0%|          | 0/600 [00:00<?, ?it/s]

/tmp/ipykernel_35058/3565486690.py:32: FutureWarning: GLiNER.batch_predict_entities is deprecated and will be removed in a future release. Please use GLiNER.inference instead.
  batch_preds = model.batch_predict_entities(


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

...

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

💾 Test 결과 저장 완료! 경로: ./document_list/test_ner_hybrid.json


In [ ]:
from tqdm.notebook import tqdm  # 코랩용 예쁜 진행바
import json
import numpy as np
import os

# 결과를 담을 리스트
result_for_test = []

print("🚀 Validation 데이터 전체 평가 시작...")

# 1. 데이터셋 루프
for i in tqdm(range(len(valid_dataset['validation']))):

    # 질문과 id 가져오기
    test_q_query = valid_dataset['validation'][i]['question']
    test_q_id = valid_dataset['validation'][i]['id']

    # 2. 골든리트리버 (Fusion 검색)
    retrieved_nodes_test = gliner_bm25_retriever.retrieve(test_q_query)

    # 3. Reranker용 데이터 준비
    docs_for_rerank_test = [n.node.text for n in retrieved_nodes_test]
    # metadata 안전하게 가져오기 (혹시 모를 에러 방지용 get 사용)
    ids_for_rerank_test = [n.node.metadata.get('document_id', -1) for n in retrieved_nodes_test]

    # 4. Rerank 실행 (중복 제거 로직 포함됨)
    # reranked_results_test[1] 에는 이미 중복이 제거된 ID 리스트가 들어있습니다.
    reranked_results_test = reranker.rerank(test_q_query, docs_for_rerank_test, ids_for_rerank_test, top_k=5)

    # 5. [핵심 수정] ID 추출 (Numpy 슬라이싱 삭제) 🛠️
    # 이미 1차원 리스트이므로 그냥 int로 변환만 하면 됩니다.
    final_doc_ids = [int(did) for did in reranked_results_test[1]]

    result_for_test.append([test_q_id, final_doc_ids])

# ---------------------------------------------------------
# JSON 변환 및 저장
# ---------------------------------------------------------
def convert_to_json(data):
    question_ids = []
    document_lists = []

    for q_id, doc_list in data:
        question_ids.append(q_id)
        document_lists.append(doc_list)

    result_dict = {
        "question_id": question_ids,
        "document_id": document_lists
    }
    return result_dict

# 변환
json_test = convert_to_json(result_for_test)

# 파일 경로 (요청하신 경로)
file_path = './document_list/validation_ner_only.json'

# 폴더가 없으면 에러 나니까 생성해주는 코드 추가
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# 저장 (한글 깨짐 방지 포함)
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(json_test, f, ensure_ascii=False, indent=4)

print(f"💾 저장 완료! 경로: {file_path}")

🚀 Validation 데이터 전체 평가 시작...


  0%|          | 0/240 [00:00<?, ?it/s]

/tmp/ipykernel_35058/3565486690.py:32: FutureWarning: GLiNER.batch_predict_entities is deprecated and will be removed in a future release. Please use GLiNER.inference instead.
  batch_preds = model.batch_predict_entities(


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

...

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

💾 저장 완료! 경로: ./document_list/validation_ner_only.json


In [ ]:
from tqdm.notebook import tqdm
import json
import numpy as np
import os

# 🚨 결과 담을 리스트 (이름 통일함)
final_test_results = []

print("🚀 Test 데이터셋 전체 평가 시작...")

# 1. 데이터셋 루프 (valid -> test 로 변경)
for i in tqdm(range(len(test_dataset['validation']))):

    # 질문과 id 가져오기 (test_dataset 사용)
    target_query = test_dataset['validation'][i]['question']
    target_id = test_dataset['validation'][i]['id']

    # 2. 골든리트리버 (Fusion 검색)
    retrieved_nodes = gliner_bm25_retriever.retrieve(target_query)

    # 3. Reranker용 데이터 준비
    docs_for_rerank = [n.node.text for n in retrieved_nodes]
    # metadata 안전하게 가져오기
    ids_for_rerank = [n.node.metadata.get('document_id', -1) for n in retrieved_nodes]

    # 4. Rerank 실행 (중복 제거 로직 포함)
    reranked_results = reranker.rerank(target_query, docs_for_rerank, ids_for_rerank, top_k=5)

    # 5. ID 추출 (reranked_results[1]이 이미 정제된 ID 리스트임)
    final_doc_ids = [int(did) for did in reranked_results[1]]

    # 🚨 위에서 선언한 리스트에 정확히 append
    final_test_results.append([target_id, final_doc_ids])

# ---------------------------------------------------------
# JSON 변환 및 저장
# ---------------------------------------------------------
def convert_to_json(data):
    question_ids = []
    document_lists = []

    for q_id, doc_list in data:
        question_ids.append(q_id)
        document_lists.append(doc_list)

    result_dict = {
        "question_id": question_ids,
        "document_id": document_lists
    }
    return result_dict

# 변환 (올바른 리스트 전달)
json_output = convert_to_json(final_test_results)

# 파일 경로
file_path = './document_list/test_ner_only.json'

# 폴더 생성 (안전장치)
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# 저장
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(json_output, f, ensure_ascii=False, indent=4)

print(f"💾 Test 결과 저장 완료! 경로: {file_path}")

🚀 Test 데이터셋 전체 평가 시작...


  0%|          | 0/600 [00:00<?, ?it/s]

/tmp/ipykernel_35058/3565486690.py:32: FutureWarning: GLiNER.batch_predict_entities is deprecated and will be removed in a future release. Please use GLiNER.inference instead.
  batch_preds = model.batch_predict_entities(


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

...

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

💾 Test 결과 저장 완료! 경로: ./document_list/test_ner_only.json


In [ ]:
import pickle

# 🚨 [중요] 이번엔 까먹지 말고 바로 저장합시다!
save_path = "/data/ephemeral/home/mcr/gliner_bm25_index_v1"
os.makedirs(save_path, exist_ok=True)

with open(f"{save_path}/nodes.pkl", "wb") as f:
    pickle.dump(nodes, f)

print("💾 Nodes 파일 영구 저장 완료! (이제 꺼져도 안전함)")

💾 Nodes 파일 영구 저장 완료! (이제 꺼져도 안전함)


결과물 확인용 코드입니다.

In [ ]:
### 백업용

import bm25s
import pickle
import os
from functools import partial
from gliner import GLiNER

# =========================================================
# 1. 준비물: GLiNER 모델은 다시 로드해야 함 (이건 어쩔 수 없음)
# =========================================================
# (인덱싱을 다시 하는 게 아니라, 질문을 이해하기 위해 모델만 메모리에 올리는 과정)
print("🚀 GLiNER 모델 로드 중...")
gliner_model = GLiNER.from_pretrained("urchade/gliner_large-v2.1") # 쓰시던 모델명
gliner_model.to("cuda")

# 라벨 리스트도 다시 정의 필요
gliner_labels = [...] # 아까 쓰셨던 그 라벨 리스트 그대로

# 토크나이저 함수 다시 조립
my_gliner_batch_tokenizer = partial(
    tokenize_gliner_batch, # 아까 정의한 그 함수
    model=gliner_model,
    labels=gliner_labels,
    label_chunk_size=50,
    score_threshold_ratio=2.5
)

# =========================================================
# 2. 저장된 파일로 "검색기(Body)" 합체!
# =========================================================
save_dir = "gliner_bm25_index_v1" # 저장했던 폴더 경로

print(f"📂 '{save_dir}'에서 GLiNER 검색기 복구 중...")

# A. 본문(Nodes) 로드
with open(f"{save_dir}/nodes.pkl", "rb") as f:
    loaded_nodes = pickle.load(f)

# B. 인덱스(BM25) 로드 (여기가 9시간 절약 구간!!)
loaded_bm25 = bm25s.BM25.load(save_dir, load_corpus=False)

# C. 합체 (Indexing 과정 스킵)
restored_retriever = GLiNerBM25Retriever.__new__(GLiNerBM25Retriever)

restored_retriever._nodes = loaded_nodes      # 본문 주입
restored_retriever._bm25 = loaded_bm25        # 뇌(인덱스) 주입
restored_retriever._tokenizer = my_gliner_batch_tokenizer # 질문 처리기 주입
restored_retriever._similarity_top_k = 30
restored_retriever.doc_batch_size = 32



In [29]:
# 기존 함수에 return_tags 옵션만 추가해서 덮어쓰기!
def tokenize_gliner_batch(
    texts: List[str],
    model: GLiNER,
    labels: List[str],
    label_chunk_size: int = 20,
    score_threshold_ratio: float = 1.05,
    return_tags: bool = False  # 🚨 옵션 추가
):
    if not texts: return []

    # --- (기존 로직 복붙) ---
    batch_results = [[] for _ in texts]
    for i in range(0, len(labels), label_chunk_size):
        sub_labels = labels[i : i + label_chunk_size]
        try:
            batch_preds = model.batch_predict_entities(
                texts, sub_labels, flat_ner=True, threshold=0.1
            )
            for doc_idx, entities in enumerate(batch_preds):
                batch_results[doc_idx].extend(entities)
        except Exception: continue

    final_batch_tokens = []
    for entities in batch_results:
        if not entities:
            final_batch_tokens.append([])
            continue

        max_score = max(e['score'] for e in entities)
        cutoff_score = max_score / score_threshold_ratio
        filtered = [e for e in entities if e['score'] >= cutoff_score]

        doc_tokens = []
        seen_texts = set()

        for e in filtered:
            token_text = e['text']
            token_label = e['label'] # 라벨 가져오기

            tokens_to_add = []
            if token_text.count(' ') >= 2:
                for t in token_text.split():
                    tokens_to_add.append((t, token_label))
            else:
                tokens_to_add.append((token_text, token_label))

            for txt, lbl in tokens_to_add:
                if txt not in seen_texts:
                    seen_texts.add(txt)
                    # 🚨 태그 반환 로직
                    if return_tags:
                        doc_tokens.append((txt, lbl))
                    else:
                        doc_tokens.append(txt)

        final_batch_tokens.append(doc_tokens)

    return final_batch_tokens

In [44]:
def inspect_query_without_reindexing(retriever, query_str):
    """
    기존 리트리버를 뜯어서 쿼리 분석을 수행하고,
    👉 [결과 리스트를 반환]합니다.
    """
    print(f"🔍 Query Inspection: {query_str}")

    # 1. 재료 꺼내기
    frozen_tokenizer = retriever._tokenizer
    captured_model = frozen_tokenizer.keywords.get('model')
    captured_labels = frozen_tokenizer.keywords.get('labels')
    captured_chunk_size = frozen_tokenizer.keywords.get('label_chunk_size', 20)
    captured_threshold = frozen_tokenizer.keywords.get('score_threshold_ratio', 1.05)

    if not captured_model:
        print("😱 모델을 찾을 수 없습니다.")
        return []

    # 2. 분석 실행 (경고 메시지 끄기 포함)
    import warnings
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning) # 빨간 경고 무시

        results = tokenize_gliner_batch(
            texts=[query_str],
            model=captured_model,
            labels=captured_labels,
            label_chunk_size=captured_chunk_size,
            score_threshold_ratio=captured_threshold,
            return_tags=True # 태그 포함
        )[0]

    # 3. 출력
    if not results:
        print("    추출된 토큰이 없습니다.")
    else:
        print(f"    추출된 토큰 ({len(results)}개):")
        for token, label in results:
            print(f"      - [{label}] {token}")

    # 🚨 [핵심] 결과를 밖으로 던져줍니다!
    return results

In [45]:
# 예: retriever 변수명이 gliner_bm25_retriever 라고 가정
query = "이순신 장군이 거북선을 만든 곳은?"

inspect_query_without_reindexing(restored_retriever, query)

🔍 Query Inspection: 이순신 장군이 거북선을 만든 곳은?
    추출된 토큰 (3개):
      - [문명_직위/직책] 장군
      - [인물_사람] 이순신
      - [인공물_교통수단/운송수단] 거북선


[('장군', '문명_직위/직책'), ('이순신', '인물_사람'), ('거북선', '인공물_교통수단/운송수단')]

In [48]:
# 1. 확인할 질문 가져오기
query = dataset['train'][153]['question']

print(f" 원본 질문: {query}")
print("-" * 50)

# 2. 분석 함수 호출 (이제 값을 반환받습니다!)
# inspect 함수는 단일 쿼리를 처리하므로 리스트로 감쌀 필요 없이 바로 넣습니다.
extracted_tokens = inspect_query_without_reindexing(restored_retriever, query)

print("-" * 50)

# 3. 결과 확인
# (extracted_tokens는 [('장군', '직위'), ('이순신', '인물')] 형태의 리스트입니다)

if not extracted_tokens:
    print(" [경고] 토큰이 하나도 없습니다! (검색 결과 0건 확정)")
    print("   -> GLiNer가 이 질문에서 중요한 단어를 못 찾았습니다.")
else:
    print(f" GLiNer가 뽑은 토큰 리스트:")
    print(extracted_tokens)


 원본 질문: KT가 '쇼옴니아' 명칭을 사용하지 못하는 계기가 된 핸드폰은 어느 회사 제품인가?
--------------------------------------------------
🔍 Query Inspection: KT가 '쇼옴니아' 명칭을 사용하지 못하는 계기가 된 핸드폰은 어느 회사 제품인가?
    추출된 토큰 (3개):
      - [기관_경제] KT
      - [인공물_기타 상품] 옴니아
      - [용어_IT 하드웨어] 핸드폰
--------------------------------------------------
 GLiNer가 뽑은 토큰 리스트:
[('KT', '기관_경제'), ('옴니아', '인공물_기타 상품'), ('핸드폰', '용어_IT 하드웨어')]
